In [1]:
pip install opencv-python

In [2]:
pip install face_recognition

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install cvzone

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install firebase_admin

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import os
import pickle
import face_recognition
import numpy as np
import cvzone
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from firebase_admin import storage
from datetime import datetime

cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred,{
    'databaseURL':"https://face-recognitionattendance-default-rtdb.firebaseio.com/",
    'storageBucket':"face-recognitionattendance.appspot.com"
})

bucket = storage.bucket()
#capturing and detetcing the live image of person from webcam
cap = cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)

imgBackground = cv2.imread("Resources/background.png")

folderModesPath = "Resources/Modes"
modesPathList = os.listdir(folderModesPath)
imgModesList = []

#importing modes images in the list

for path in modesPathList:
    imgModesList.append(cv2.imread(os.path.join(folderModesPath, path)))
#print(modesPathList)
#print(len(imgModesList))

#loading the encoding file 
print("Loading ecoded file.....")
file = open('EncodeFile.p',"rb")
encodeListKnownWithIds = pickle.load(file)
file.close()
encodeListKnown, studentsId = encodeListKnownWithIds
#print(studentsId)
print("Encoded File completed.....")

modeType = 0
counter = 0
id = -1
imgStudent = []

while True:
    success, img = cap.read()
    
    #resizing the image
    imgSize = cv2.resize(img, (0,0), None, 0.25, 0.25)
    imgSize = cv2.cvtColor(imgSize, cv2.COLOR_BGR2RGB)

    faceCurFrame = face_recognition.face_locations(imgSize)
    encodeCurFrame = face_recognition.face_encodings(imgSize, faceCurFrame)
    
    imgBackground[162:162+480, 55:55+640]= img
    imgBackground[44:44+633, 808:808+414]= imgModesList[modeType]
    
    if faceCurFrame:
        for encFaces, facLocation in zip(encodeCurFrame, faceCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encFaces)
            faceDist = face_recognition.face_distance(encodeListKnown, encFaces)

    #         print("matches", matches)

    #         print("faceDist", faceDist)

            matchIndex = np.argmin(faceDist)
            #print("Match Index", matchIndex)

            if matches[matchIndex]:
    #             print("Known Face Detected....")
    #             print(studentsId[matchIndex]) #to tell which student image id has been detected.
                y1, x2, y2, x1 = facLocation
                y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4      
                bbox = 55+x1, 162+y1, x2-x1, y2-y1
                imgBackround = cvzone.cornerRect(imgBackground, bbox, rt = 0)

                id = studentsId[matchIndex]
                if counter == 0:
                    counter = 1
                    modeType = 1
            if counter != 0:
                if counter == 1:
                    studentInfo = db.reference(f'Students/{id}').get()
                    print(studentInfo)
                    blob = bucket.get_blob(f'Images/{id}.png')
                    array = np.frombuffer(blob.download_as_string(),np.uint8)
                    imgStudent = cv2.imdecode(array, cv2.COLOR_BGRA2BGR)
                    #update the data attendance
                    datetimeObject = datetime.strptime(studentInfo['Attendance_Time'],"%Y-%m-%d %H:%M:%S")
                    secondsElapsed = (datetime.now()-datetimeObject).total_seconds()
                    print(secondsElapsed)
                    if secondsElapsed > 30:

                        ref = db.reference(f'Students/{id}')
                        studentInfo['Total_Attendance'] += 1

                        #update the data in our database or server
                        ref.child("Total_Attendance").set(studentInfo['Total_Attendance'])
                        #update the last attendance time
                        ref.child("Attendance_Time").set(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                    else:
                        modeType = 3
                        counter = 0
                        imgBackground[44:44+633, 808:808+414]= imgModesList[modeType]


                if modeType != 3:


                    if 10 < counter < 20:
                        modeType = 2
                    imgBackground[44:44+633, 808:808+414]= imgModesList[modeType]


                    if counter <= 10:

                        cv2.putText(imgBackground,str(studentInfo['Total_Attendance']),(861, 125),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1)

                        #inclining name in the centre
                        (w,h), _ = cv2.getTextSize(studentInfo['name'],cv2.FONT_HERSHEY_COMPLEX, 1,1)
                        offset = (414-w)//2
                        cv2.putText(imgBackground,str(studentInfo['name']),(808+offset, 445),cv2.FONT_HERSHEY_COMPLEX,1,(50,50,50),1)

                        cv2.putText(imgBackground,str(studentInfo['major']),(1006, 550),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),1)
                        cv2.putText(imgBackground,str(id),(1006, 493),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),1)

                        #imgBackground[507:507+213, 375:375+216] = imgStudent
                    counter += 1

                    if counter >= 20:
                        counter = 0
                        modeType = 0
                        studentInfo =[]
                        imgStudent = []
                        imgBackground[44:44+633, 808:808+414]= imgModesList[modeType]
    
    else:
        modeType = 0
        counter = 0
    #cv2.imshow("Webcam", img)
    cv2.imshow("Face Attendance", imgBackground)
    cv2.waitKey(1)

Loading ecoded file.....
Encoded File completed.....
{'Attendance_Time': '2024-04-19 00:54:34', 'Graduated_Year': 2025, 'Starting_Year': 2021, 'Total_Attendance': 1, 'major': 'Btech', 'name': 'Bhavika Garg'}
394837.302053
{'Attendance_Time': '2024-04-23 14:35:11', 'Graduated_Year': 2025, 'Starting_Year': 2021, 'Total_Attendance': 2, 'major': 'Btech', 'name': 'Bhavika Garg'}
7.220561
{'Attendance_Time': '2024-04-23 14:35:11', 'Graduated_Year': 2025, 'Starting_Year': 2021, 'Total_Attendance': 2, 'major': 'Btech', 'name': 'Bhavika Garg'}
12.358353
{'Attendance_Time': '2024-04-23 14:35:11', 'Graduated_Year': 2025, 'Starting_Year': 2021, 'Total_Attendance': 2, 'major': 'Btech', 'name': 'Bhavika Garg'}
22.727507
{'Attendance_Time': '2024-04-23 14:35:11', 'Graduated_Year': 2025, 'Starting_Year': 2021, 'Total_Attendance': 2, 'major': 'Btech', 'name': 'Bhavika Garg'}
27.721876
{'Attendance_Time': '2024-04-23 14:35:11', 'Graduated_Year': 2025, 'Starting_Year': 2021, 'Total_Attendance': 2, 'major